In [1]:
%%capture
%load_ext sql
%sql sqlite:///C:/Users/14056/Documents/chinook.db

In [2]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
#Our first task is to gather some information on a specific purchase. 
#For one single purchase (invoice_id) we want to know, for each track purchased:

#The id of the track.
#The name of the track.
#The name of media type of the track.
#The price that the customer paid for the track.
#The quantity of the track that was purchased.

In [5]:
%%sql
SELECT * FROM invoice_line
WHERE invoice_id = 3;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
27,3,2516,0.99,1
28,3,2646,0.99,1


In [6]:
%%sql
SELECT * FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
WHERE invoice_id = 3;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,track_id_1,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_1
27,3,2516,0.99,1,2516,Black Hole Sun,203,1,1,Soundgarden,320365,10425229,0.99
28,3,2646,0.99,1,2646,I Looked At You,214,1,1,"Robby Krieger, Ray Manzarek, John Densmore, Jim Morrison",142080,4663988,0.99


In [7]:
%%sql

SELECT * FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE invoice_id = 3;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,track_id_1,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_1,media_type_id_1,name_1
27,3,2516,0.99,1,2516,Black Hole Sun,203,1,1,Soundgarden,320365,10425229,0.99,1,MPEG audio file
28,3,2646,0.99,1,2646,I Looked At You,214,1,1,"Robby Krieger, Ray Manzarek, John Densmore, Jim Morrison",142080,4663988,0.99,1,MPEG audio file


In [8]:
%%sql
SELECT
    il.track_id,
    t.name track_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE invoice_id = 4
LIMIT 5;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


track_id,track_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",Protected AAC audio file,0.99,1
2560,Violent Pornography,MPEG audio file,0.99,1
3336,War Pigs,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,MPEG audio file,0.99,1
1872,Attitude,MPEG audio file,0.99,1


In [9]:
#Let's extend the query we wrote in the previous screen by adding the artist for each track

In [10]:
%%sql
SELECT
    il.invoice_id,
    il.track_id,
    t.name track_name,
    ar.name artist_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar on ar.artist_id = al.artist_id
LIMIT 5;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


invoice_id,track_id,track_name,artist_name,track_type,unit_price,quantity
1,1158,Right Next Door to Hell,Guns N' Roses,Protected AAC audio file,0.99,1
1,1159,Dust N' Bones,Guns N' Roses,Protected AAC audio file,0.99,1
1,1160,Live and Let Die,Guns N' Roses,Protected AAC audio file,0.99,1
1,1161,Don't Cry (Original),Guns N' Roses,Protected AAC audio file,0.99,1
1,1162,Perfect Crime,Guns N' Roses,Protected AAC audio file,0.99,1


In [11]:
#what we want to produce is a query that lists the top 10 artists, 
#calculated by the number of times a track by that artist has been purchased.

In [12]:
%%sql
SELECT
    t.track_id,
    ar.name artist_name
FROM track t
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
ORDER BY 1 LIMIT 5;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


track_id,artist_name
1,AC/DC
2,Accept
3,Accept
4,Accept
5,Accept


In [13]:
#we need to join this subquery to our invoice_line table.

In [14]:
%%sql
SELECT
    il.invoice_line_id,
    il.track_id,
    ta.artist_name
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
ORDER BY 1 LIMIT 5;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


invoice_line_id,track_id,artist_name
1,1158,Guns N' Roses
2,1159,Guns N' Roses
3,1160,Guns N' Roses
4,1161,Guns N' Roses
5,1162,Guns N' Roses


In [15]:
#calculate the number of times each artist has had a track purchased, and find the top 10.

In [16]:
%%sql
SELECT
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


artist,tracks_purchased
Queen,192
Jimi Hendrix,187
Red Hot Chili Peppers,130
Nirvana,130
Pearl Jam,129


In [17]:

#Write a query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased

In [18]:
%%sql
SELECT
    ta.album_title album,
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                al.title album_title,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1, 2
ORDER BY 3 DESC LIMIT 5;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


In [19]:
#query that returns information about each employee and their supervisor.
#Relationship

In [20]:
%%sql
SELECT
    e1.employee_id,
    e2.employee_id supervisor_id
FROM employee e1
INNER JOIN employee e2 on e1.reports_to = e2.employee_id
LIMIT 4;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


employee_id,supervisor_id
2,1
6,1
3,2
4,2


In [21]:
%%sql
SELECT
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name,
    e2.title supervisor_title
FROM employee e1
LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
ORDER BY 1;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


employee_name,employee_title,supervisor_name,supervisor_title
Andrew Adams,General Manager,None,None
Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
Laura Callahan,IT Staff,Michael Mitchell,IT Manager
Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
Michael Mitchell,IT Manager,Andrew Adams,General Manager
Nancy Edwards,Sales Manager,Andrew Adams,General Manager
Robert King,IT Staff,Michael Mitchell,IT Manager
Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


In [22]:
#Write a query that finds the contact details of a customer with a first_name containing jen from the database.
#Write a query that finds the contact details of a customer with a first_name containing belle from the database.

In [23]:
%%sql
SELECT
    first_name,
    last_name,
    phone
FROM customer
WHERE first_name LIKE "Jen%";

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


first_name,last_name,phone
Jennifer,Peterson,+1 (604) 688-2255


In [24]:
%%sql
SELECT
    first_name,
    last_name,
    phone
FROM customer
where first_name LIKE "%belle%";

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


first_name,last_name,phone
Isabelle,Mercier,+33 03 80 73 66 99


In [25]:
#SQLite LIKE is case insensitive, so LIKE "%jen%" will match Jen and JEN and JeN. Other flavors of SQL may be case sensitive,
#so you may need to use the LOWER() function to get a case insensitive match.

In [26]:
# categorization to your query five different media type string Protected in the name column. 
# Any rows with a match get a value of 1; all other rows get 0

In [27]:
%%sql
SELECT
    media_type_id,
    name,
    CASE
        WHEN name LIKE '%Protected%' THEN 1
        ELSE 0
        END
        AS protected
FROM media_type;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


media_type_id,name,protected
1,MPEG audio file,0
2,Protected AAC audio file,1
3,Protected MPEG-4 video file,1
4,Purchased AAC audio file,0
5,AAC audio file,0


In [28]:
#Write a query that summarizes the purchases of each customer based on spending category

In [29]:
%%sql
SELECT
   c.first_name || " " || c.last_name customer_name,
   COUNT(i.invoice_id) number_of_purchases,
   SUM(i.total) total_spent,
   CASE
       WHEN sum(i.total) < 40 THEN 'small spender'
       WHEN sum(i.total) > 100 THEN 'big spender'
       ELSE 'regular'
       END
       AS customer_category
FROM invoice i
INNER JOIN customer c ON i.customer_id = c.customer_id
GROUP BY 1 
ORDER BY 1
LIMIT 4;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


customer_name,number_of_purchases,total_spent,customer_category
Aaron Mitchell,8,70.28999999999999,regular
Alexandre Rocha,10,69.3,regular
Astrid Gruber,9,69.3,regular
Bjørn Hansen,9,72.27000000000001,regular


In [30]:
%%sql
SELECT
    ta.artist_name artist,
    COUNT(*) tracks_sold
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC LIMIT 10;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


artist,tracks_sold
Queen,192
Jimi Hendrix,187
Red Hot Chili Peppers,130
Nirvana,130
Pearl Jam,129
Guns N' Roses,124
AC/DC,124
Foo Fighters,121
The Rolling Stones,117
Metallica,106


In [31]:
# Write a query that summarizes the ALBUM NAME  Jagged Little Pill

In [32]:
%%sql
WITH track_info AS
    (                
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )

SELECT * FROM track_info
WHERE album_name = "Jagged Little Pill";

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


name,artist,album_name,media_type,genre,length_milliseconds
All I Really Want,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,284891
You Oughta Know,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,249234
Perfect,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,188133
Hand In My Pocket,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,221570
Right Through You,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,176117
Forgiven,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,300355
You Learn,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,239699
Head Over Feet,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,267493
Mary Jane,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,280607
Ironic,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,229825


In [33]:
#Create a query that shows summary data for every playlist in the Chinook database

In [34]:
%%sql
WITH playlist_info AS
    (
     SELECT
         p.playlist_id,
         p.name playlist_name,
         t.name track_name,
         (t.milliseconds / 1000) length_seconds
     FROM playlist p
     LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
     LEFT JOIN track t ON t.track_id = pt.track_id
    )

SELECT
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_info
GROUP BY 1, 2
ORDER BY 1;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,876049
2,Movies,0,None
3,TV Shows,213,500987
4,Audiobooks,0,None
5,90’s Music,1477,397970
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,876049
9,Music Videos,1,294
10,TV Shows,213,500987


In [35]:
#Create a query that shows summary data for every playlist name genric

In [36]:
%%sql
WITH playlist_info AS
    (
     SELECT
         p.playlist_id,
         p.name playlist_name,
         t.name track_name,
         (t.milliseconds / 1000) length_seconds
     FROM playlist p
     LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
     LEFT JOIN track t ON t.track_id = pt.track_id
    )

SELECT
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_info
GROUP BY 1
ORDER BY 1;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


playlist_name,number_of_tracks,length_seconds
90’s Music,1477,397970
Audiobooks,0,None
Brazilian Music,39,9464
Classical,75,21736
Classical 101 - Deep Cuts,25,6742
Classical 101 - Next Steps,25,7565
Classical 101 - The Basics,25,7429
Grunge,15,4114
Heavy Metal Classic,26,8189
Movies,0,None


In [37]:
#a subquery that we can use again and again.
# writing Query that customer in the USA only

In [38]:
%%sql
CREATE VIEW customer_usa AS 
     SELECT * FROM customer
     WHERE country = "USA";

 * sqlite:///C:/Users/14056/Documents/chinook.db
(sqlite3.OperationalError) table customer_usa already exists
[SQL: CREATE VIEW customer_usa AS 
     SELECT * FROM customer
     WHERE country = "USA";]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [39]:
%%sql
SELECT * FROM customer_usa;


 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
24,Frank,Ralston,None,162 E Superior Street,Chicago,IL,USA,60611,+1 (312) 332-3232,None,fralston@gmail.com,3
25,Victor,Stevens,None,319 N. Frances Street,Madison,WI,USA,53703,+1 (608) 257-0597,None,vstevens@yahoo.com,5


In [40]:
#Create a view contain the columns from customers in their original order.
#The view should contain only customers who have purchased more than $90 in tracks from the store.

In [41]:
%%sql
CREATE VIEW customer_gt_90_dollars AS
SELECT
        c.*
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90;
    SELECT * FROM customer_gt_90_dollars;

 * sqlite:///C:/Users/14056/Documents/chinook.db
(sqlite3.OperationalError) table customer_gt_90_dollars already exists
[SQL: CREATE VIEW customer_gt_90_dollars AS
SELECT
        c.*
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [42]:
#Use UNION to produce a table of customers in the USA or have spent more than $90,

In [43]:
%%sql
SELECT * from  customer_usa

UNION

SELECT * from  customer_gt_90_dollars;
SELECT customer_usa.city from  customer_usa
UNION
SELECT customer_gt_90_dollars.city from  customer_gt_90_dollars;



 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.
Done.


city
Bordeaux
Boston
Brasília
Chicago
Cupertino
Delhi
Dublin
Fort Worth
Frankfurt
Lisbon


In [44]:
#Use UNION to produce unique cities either customers in the USA or have spent more than $90

In [45]:
%%sql
SELECT customer_usa.city from  customer_usa
UNION
SELECT customer_gt_90_dollars.city from  customer_gt_90_dollars;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


city
Bordeaux
Boston
Brasília
Chicago
Cupertino
Delhi
Dublin
Fort Worth
Frankfurt
Lisbon


In [46]:
# Customers who are in the USA and have spent more than $90

In [47]:
%%sql
SELECT * from customer_usa

INTERSECT

SELECT * from customer_gt_90_dollars;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4


In [48]:
#Identifying customers who are in the USA and have not spent $90 

In [49]:
%%sql
SELECT * from customer_usa

EXCEPT

SELECT * from customer_gt_90_dollars;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
24,Frank,Ralston,None,162 E Superior Street,Chicago,IL,USA,60611,+1 (312) 332-3232,None,fralston@gmail.com,3
25,Victor,Stevens,None,319 N. Frances Street,Madison,WI,USA,53703,+1 (608) 257-0597,None,vstevens@yahoo.com,5
26,Richard,Cunningham,None,2211 W Berry Street,Fort Worth,TX,USA,76110,+1 (817) 924-7272,None,ricunningham@hotmail.com,4
27,Patrick,Gray,None,1033 N Park Ave,Tucson,AZ,USA,85719,+1 (520) 622-4200,None,patrick.gray@aol.com,4
28,Julia,Barnett,None,302 S 700 E,Salt Lake City,UT,USA,84102,+1 (801) 531-7272,None,jubarnett@gmail.com,5


In [50]:
#Write a query that works out how many customers that are in the USA and 
#have purchased more than $90 are assigned to each sales support agent

In [51]:
%%sql
WITH customers_usa_gt_90 AS
    (
     SELECT * FROM customer_usa

     INTERSECT

     SELECT * FROM customer_gt_90_dollars
    )

SELECT
    e.first_name || " " || e.last_name employee_name,
    COUNT(c.customer_id) customers_usa_gt_90
FROM employee e
LEFT JOIN customers_usa_gt_90 c ON c.support_rep_id = e.employee_id
WHERE e.title = 'Sales Support Agent'
GROUP BY 1 ORDER BY 1;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


employee_name,customers_usa_gt_90
Jane Peacock,0
Margaret Park,2
Steve Johnson,2


In [52]:
#Write a query that uses multiple named subqueries in a WITH clause to gather total sales data on customers from India:
#The first named subquery should return all customers that are from India.
#The second named subquery should calculate the sum total for every customer.
#You can also use WHERE insted of multiple subquries

In [53]:
%%sql

WITH
    customers_india AS
        (
        SELECT * FROM customer
        WHERE country = "India"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id,
             SUM(total) total
         FROM invoice
         GROUP BY 1
        )

SELECT
    ci.first_name || " " || ci.last_name customer_name,
    spc.total total_purchases
FROM customers_india ci
INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id
ORDER BY 1;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


customer_name,total_purchases
Manoj Pareek,111.86999999999999
Puja Srivastava,71.28


In [54]:
#Create a query to find the customer from each country that has spent the most money at our store, 
#ordered alphabetically by country.

In [55]:
%%sql
WITH
    customer_country_purchases AS
        (
         SELECT
             i.customer_id,
             c.country,
             SUM(i.total) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY 1, 2
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY 1
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    cbc.max_purchase total_purchased
FROM customer c
INNER JOIN country_best_customer cbc ON cbc.customer_id = c.customer_id
ORDER BY 1 ASC

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.38999999999999
Brazil,Luís Gonçalves,108.89999999999998
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02000000000001
Czech Republic,František Wichterlová,144.54000000000002
Denmark,Kara Nielsen,37.61999999999999
Finland,Terhi Hämäläinen,79.2


In [56]:
#Write a query that returns each genre, with the number of tracks sold in the USA:
#in absolute numbers and in percentages.

In [57]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In [58]:
#Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company.

In [59]:
%%sql

WITH customer_support_rep_sales AS
    (
     SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1,2
    )

SELECT
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    SUM(csrs.total) total_sales
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON e.employee_id = csrs.support_rep_id
GROUP BY 1;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


In [60]:
#While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee),
#the difference roughly corresponds with the differences in their hiring dates.

In [61]:
#Write a query that collates data on purchases from different countries.
#Where a country has only one customer, collect them into an "Other" group.
#The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
#For each country, include:
#total number of customers
#total value of sales
#average value of sales per customer
#average order value

In [62]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [63]:
#Based on the data, there may be opportunity in the following countries:Czech Republic United Kingdom India
#It's worth keeping in mind that because the amount of data from each of these countries is relatively low.
#Because of this, we should be cautious spending too much money on new marketing campaigns, 
#as the sample size is not large enough to give us high confidence. 
#A better approach would be to run small campaigns in these countries,
#collecting and analyzing the new customers to make sure that these trends hold with new customers.

In [64]:
#Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
#Number of invoices
#Percentage of invoices

In [65]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///C:/Users/14056/Documents/chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


In [66]:
#Albums vs Individual Tracks
#Album purchases account for 18.6% of purchases
#Based on this data, I would recommend against purchasing only select tracks from albums from record companies
#since there is potential to lose one fifth of revenue